### Step 1: Authenticating

Run the cell below (Imports) to load all functions required for the compilation.

You will be prompted by Google to authenticate and allow access to your Google Drive. Follow the instructions.

In [ ]:
#@markdown Imports

import math
import numpy as np
import pandas as pd
import requests
import json
import seaborn as sns; sns.set()
import ipywidgets as widgets
from IPython.display import HTML
import gspread
from gspread_dataframe import set_with_dataframe
import matplotlib.pyplot as plt
from collections import Counter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')


def get_votes_df(votesheet, finalchar="", **options):

  votes = np.array(votesheet.get_all_values())
  votes_df = pd.DataFrame(votes[1:,:], columns=votes[0], index=range(1,len(votes[:,0])))

  # Replace non-breaking white space
  votes_df = votes_df.replace(u'\u00A0',' ', regex=True)

  # Tag unranked votes
  for voter in votes_df:
    votes_df[voter] = votes_df[voter].mask(votes_df[voter].str.startswith("["), "-1. " + votes_df[voter].astype(str))
    
    #remove comments after finalchar
    if finalchar != "":
      vvv = votes_df[voter].str.split("]", n=1, expand=True) + "]"
      votes_df[voter] = vvv[0]
      
  # Replace non-default delimiters
  for sd in options["special_delimiters"]:
    votes_df.replace(sd, options["default_delimiter"], regex=True, inplace=True)

  # Take care of whitespace inconsistencies
  if options["vote_type"] == "ID":
    votes_df = votes_df.replace(' ','', regex=True).replace('.\[','. [', regex=True)
  return votes_df.mask(votes_df=="]", "0")

def get_vote_matrix(votes_df):

  vote_matrix = pd.DataFrame()

  for voter in votes_df:
    for i, vote in enumerate(votes_df[voter]):
      if vote == "": break
      try: rank, title = vote.split(". ", 1)
      except ValueError: 
        rank, title = i+1, vote.split(". ", 1)[-1]
      title = title.lstrip()
      vote_matrix.loc[title, voter] = rank

  vote_matrix.fillna(0,inplace=True)
  try:
    vote_matrix = vote_matrix.astype(pd.SparseDtype("int", 0))
  except ValueError:
    for col in vote_matrix:
      try:
        vote_matrix[col].astype(int)
      except ValueError:
        print(col, "failed")
        for row in vote_matrix[col].index:
          try:
            int(vote_matrix.loc[row,col])
          except ValueError:
            print("    rank", vote_matrix.loc[row,col], "failed")
            continue
        else:
          raise Exception(f"Check sheet again for: {col}")
      print(col, "succeeded")

  try: vote_matrix = vote_matrix.drop(["0"])
  except KeyError: pass
  #print('Density:', vote_matrix.sparse.density, '\nvote_matrix.shape', vote_matrix.shape)
  return vote_matrix

def get_titles_df(metasheet):

  titles_arr = np.array(metasheet.get_all_values())
  titles_df = pd.DataFrame(titles_arr[1:,1], index=titles_arr[1:,0], columns=["Title"])
  return titles_df

def get_meta_df(metasheet):

  titles_arr = np.array(metasheet.get_all_values())
  display(titles_arr)
  multiindex = [np.array(titles_arr[1:,0]),np.array(titles_arr[1:,1])]
  meta_df = pd.DataFrame(titles_arr[1:,2:], index=multiindex, columns=titles_arr[0,2:])
  meta_df.index.names = ["ID","Title"]
  return meta_df

def get_vote_matrix_titled(vote_matrix, meta_df):

  vote_matrix = vote_matrix.sort_index()
  meta_df = meta_df.sort_index(level="Title")
  v = vote_matrix.index
  new_ix = meta_df[meta_df.index.get_level_values("Title").isin(v)].index
  
  if len(new_ix) != len(v): 
    meta_df = meta_df.sort_index(level="ID")
    new_ix = meta_df[meta_df.index.get_level_values("ID").isin(v)].index

  if len(new_ix) != len(v):
      a = vote_matrix.index
      b = meta_df.index.get_level_values("Title")
      if len(set(a).difference(set(b))) > 0: display(set(a).symmetric_difference(set(b)))
      dupes = meta_df[b.duplicated(keep=False)]
      if len(dupes) > 0: display(dupes)
      raise Exception("Indices gone wild")

  vote_matrix.index = new_ix

  # vote_matrix = vote_matrix.sort_index()

  # # Case 1: vote_matrix.index consists of IDs
  # meta_df = meta_df.sort_index()

  # if np.sum(vote_matrix.index != meta_df.index.get_level_values("ID")) != 0:
  #   # Case 2: vote_matrix.index consists of titles
  #   meta_df = meta_df.sort_index(level="Title")
  #   if np.sum(vote_matrix.index != meta_df.index.get_level_values("Title")) != 0:
  #     a = vote_matrix.index
  #     b = meta_df.index.get_level_values("Title")
  #     if len(set(a).difference(set(b))) > 0:
  #       display(set(a).symmetric_difference(set(b)))
  #       raise Exception("")

  # vote_matrix.index = meta_df.index

  return vote_matrix

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def superellipse(x, n=2, a=1, b=1, size=1):
  return b * (size**n - np.abs(x/a)**n)**(1/n)

def linear_pop_multiplier(counts, most_votes, pop_weight):

  theta = np.linspace(-1/most_votes, 1/most_votes, 201)[pop_weight+100]
  b = (1-theta*most_votes)/2
  multipliers = theta * counts + b
  return 2*multipliers

def exp_pop_multiplier(counts, most_votes, pop_weight):

  if pop_weight == 0: return np.ones(len(counts))

  multipliers = 1 + most_votes * np.exp(-(counts-1)**2 / (2*(pop_weight*most_votes)**2))
  multipliers /= 1 + most_votes
  return multipliers

def elliptical_pop_multiplier(counts, most_votes, pop_weight):

  if pop_weight >= 0: # mirror superelipse along vertical axis
    counts = counts + 2 * (most_votes//2 - counts) + 1 + most_votes%2
  
  n = np.linspace(1, 0.1, 101)[np.abs(pop_weight)]
  multipliers = superellipse(counts-1, n=n, a=1, b=1/most_votes, size=most_votes) # counts-1 to move superellipse upwards
  return 2*multipliers

def get_results_df(vote_matrix, Weight, PopWeight, pop_multiplier, partial_rankings, size_dependent=False):

  for v in vote_matrix: vote_matrix[v] = vote_matrix[v].mask(vote_matrix[v]<0,partial_rankings[v].loc["Avg_Unranked_Rank"])
  list_sizes = partial_rankings.loc["Size"] if size_dependent else max(partial_rankings.loc["Size"])

  results = pd.DataFrame(index=vote_matrix.index)
  results["Votes"] = vote_matrix.astype(bool).sum(axis=1)
  MOST_VOTES = max(results["Votes"])

  score_matrix = vote_matrix.mask(vote_matrix>0, superellipse(vote_matrix-1,n=Weight,a=1,b=1,size=list_sizes)) # vm-1 to move superellipse upwards
  results["Score"] = score_matrix.sum(axis=1)

  if pop_multiplier == "vote_pop_multiplier": results["Score"] *= results["Votes"]
  else: results["Score"] *= pop_multiplier(results["Votes"], MOST_VOTES, PopWeight)

  results["Score"] = results["Score"].round(1)
  results["Score"] += 0.00001*results["Votes"] # hacky way of breaking ties by number of votes AND use method="min" for tied votes
  results["Rank"] = results["Score"].rank(ascending=False,method='min').astype(int)
  results["Score"] = results["Score"].round(1)

  return results

def get_votes_df_from_vote_matrix(vote_matrix):
  all_user_votes = []
  for v in vote_matrix: 
    all_user_votes.append(pd.Series(vote_matrix[v][vote_matrix[v] > 0].sort_values().index.get_level_values(level="Title"), name=v))
  return pd.concat(all_user_votes, axis=1)
  
def get_chart_df(vote_matrix):

  # Count votes per title
  counts = vote_matrix.astype(bool).sum(axis=1)
  cdf = pd.DataFrame(index=vote_matrix.index)
  cdf['Votes'] = counts.values
  most_votes = max(cdf['Votes'])
  max_length = np.max(vote_matrix.values)

  # Borda count
  results = get_results_df(vote_matrix, Weight=1, PopWeight=0, pop_multiplier=linear_pop_multiplier)

  # Unqiue score
  results["Unique\nScore"] = results["Score"].where(results["Votes"]==1,0)
  results["Unique\nRank"] = results["Unique\nScore"].rank(ascending=False,method='first').astype(int)

  # Popular score
  results['Popular\nScore'] = results['Score']*results['Votes']
  results["Popular\nRank"] = results["Popular\nScore"].rank(ascending=False,method='first').astype(int)

  # Gold medals
  gold_medals = get_results_df(vote_matrix, Weight=0.1, PopWeight=0, pop_multiplier=linear_pop_multiplier)
  gold_medals["Score"] /= max_length
  gold_medals.drop("Votes",axis=1, inplace=True)
  gold_medals.rename({"Score":"Gold\nMedals", "Rank":"Gold\nRank"}, axis=1, inplace=True)

  # Esoteric score
  esoteric_results = get_results_df(vote_matrix, Weight=0.4, PopWeight=-50, pop_multiplier=elliptical_pop_multiplier)
  esoteric_results.drop("Votes",axis=1, inplace=True)
  esoteric_results.rename({"Score":"Esoteric\nScore", "Rank":"Esoteric\nRank"}, axis=1, inplace=True)

  return pd.concat([results, esoteric_results, gold_medals],axis=1)

def get_list_sizes_from_vote_matrix(vote_matrix):
  """
  Needed for size-dependent Borda count
  Cannot use all_user_votes because it doesn't contain info on which lists are unranked
  """
  return vote_matrix.astype(bool).sum(axis=0)

def get_partial_rankings(vote_matrix, size_dependent=False):
  """
  Counts ballot sizes, number of ranked and unranked items, 
  avg rank of unranked items (=25.5 for completely unranked list of size 50)
  """
  if size_dependent:
    size = vote_matrix.astype(bool).sum(axis=0)
  else: 
    size = pd.Series([max(vote_matrix.max(axis=1)) for _ in range(len(vote_matrix.columns))], index=vote_matrix.columns)

  unranked = np.abs(vote_matrix.mask(vote_matrix>0,0).sum(axis=0))
  ranked = size - unranked
  avg_rank = 1+ranked+(unranked-1)/2
  df = pd.DataFrame([size.values,ranked.values,unranked.values,avg_rank.values],index=["Size","Ranked","Unranked","Avg_Unranked_Rank"],columns=size.index)
  df.loc["Avg_Unranked_Rank"] = df.loc["Avg_Unranked_Rank"].mask(df.loc["Unranked"]==0,0)
  print("rounding unranked ballots")
  df.loc["Avg_Unranked_Rank"] = df.loc["Avg_Unranked_Rank"].round(0).astype(int)
  return df

def sheet_updater(gc, SHEETNAME, verbose=False, **options):

  # Load sheets
  votesheet = gc.open(SHEETNAME).worksheet('Votes')
  metasheet = gc.open(SHEETNAME).worksheet('Titles')

  # Get votes df
  votes_df = get_votes_df(votesheet,**options)
  if verbose: display(votes_df.head())

  # Get vote matrix
  vote_matrix = get_vote_matrix(votes_df)
  if verbose: display(vote_matrix.head())
  if verbose: print("Vote matrix shape:", vote_matrix.shape)

  # Append titles to vote matrix index
  meta_df = get_meta_df(metasheet).sort_index()
  #titles_df = get_titles_df(metasheet)
  if verbose: display(meta_df.head())
  if verbose: print("Meta df shape:", meta_df.shape)

  if len(meta_df) != len(vote_matrix):   
    if verbose: print(len(meta_df),len(vote_matrix))
    print("New IDs detected:\n")
    a=set(vote_matrix.index) - set(meta_df.index.get_level_values(level=options["vote_type"]))
    b=set(meta_df.index.get_level_values(level=options["vote_type"])) - set(vote_matrix.index)
    print("Only in vote sheet:")
    for title in a: print(title)
    if verbose: print("Lenghts:", len(a),len(b))
    if verbose: print("\n\n")
    print("Only in title sheet:")
    for title in b: print(title[0] if len(b) > 1 else title)
    if verbose: display(vote_matrix.loc[a])
    if verbose: display(meta_df[meta_df.index.get_level_values(options["vote_type"]).isin(b)])
    if len(a) > 0: raise Exception("Update tiltes sheet")
    if len(b) > 0: meta_df = meta_df[~meta_df.index.get_level_values(options["vote_type"]).isin(b)]

  vote_matrix = get_vote_matrix_titled(vote_matrix, meta_df)
  # vote_matrix.to_csv(f"/content/drive/MyDrive/{SHEETNAME}_vote_matrix.csv")
  # print("Saved vote matrix:", f"/content/drive/MyDrive/{SHEETNAME}_vote_matrix.csv")
  # display(vote_matrix.head())

  # Make the chart df
  # cdf = get_chart_df(vote_matrix)

  # if update_chartsheet: 
  #   cdf["ID"] = cdf.index.get_level_values(level="ID")
  #   cdf = cdf.sort_values(by="ID")
  #   cdf["Title"] = cdf.index.get_level_values(level="Title")
  #   cols = ["Rank", "Title", "ID", "Votes",	"Score",	"Esoteric\nRank",	"Esoteric\nScore", "Gold\nRank", "Gold\nMedals", "Popular\nRank", "Popular\nScore", "Unique\nRank", "Unique\nScore"]
  #   cdf = cdf[cols]
  #   set_with_dataframe(chartsheet, cdf.sort_values("Rank"), include_index=False)
  
  return meta_df, vote_matrix
  
def load_data_from_sheet(**options):

  import gspread
  from gspread_dataframe import set_with_dataframe
  from oauth2client.client import GoogleCredentials
  #gc = gspread.authorize(GoogleCredentials.get_application_default())

  meta_df, vote_matrix = sheet_updater(gc, **options)

  if options["remove_original_title"]:

    meta_df.sort_index(level="ID",inplace=True)
    vote_matrix.sort_index(level="ID",inplace=True)

    cleaned_title = vote_matrix.index.get_level_values(level="Title")
    cleaned_title = cleaned_title.where(cleaned_title.str[-1:] != ']', cleaned_title.str[:-1].str.split('[').str[1])
    vote_matrix.index.get_level_values(level="Title")
    vote_matrix['Title'] = cleaned_title
    vote_matrix.index = vote_matrix.index.droplevel(level="Title")
    vote_matrix.set_index('Title', append=True, inplace=True)

    meta_df.index = vote_matrix.index

  nantitles = vote_matrix.index.get_level_values(level="Title").to_numpy() != vote_matrix.index.get_level_values(level="Title").to_numpy()
  vote_matrix.index = pd.MultiIndex.from_tuples([(x[0], x[0] if nan else x[1]) for x, nan in zip(vote_matrix.index, nantitles)], names=["ID","Title"])

  vote_matrix_all_ranked = vote_matrix.mask(vote_matrix < 0, 25.5)
  all_user_votes = get_votes_df_from_vote_matrix(vote_matrix_all_ranked)

  partial_rankings = get_partial_rankings(vote_matrix, size_dependent = options["size_dependent_borda"])

  if options["DEFAULT_RANK_OPTION"] =='BORDA_RANK':
    DEFAULT_RANK = get_results_df(vote_matrix, 1, 0, linear_pop_multiplier, partial_rankings, size_dependent=options["size_dependent_borda"]).sort_values(by="Rank")
  else:
    DEFAULT_RANK = get_results_df(vote_matrix, 1, 0, "vote_pop_multiplier", partial_rankings, size_dependent=options["size_dependent_borda"]).sort_values(by="Rank")

  meta_df = meta_df.loc[DEFAULT_RANK.index]

  # if query_tmdb:
  #   query_tmdb_wrapper(DEFAULT_RANK, **options)
  print("Compilation succeeded!")
  return vote_matrix, vote_matrix_all_ranked, all_user_votes, meta_df, DEFAULT_RANK

### Step 2: Enter the sheet name

Assuming you have prepared a Google sheet with all the votes according to the [sheet instructions](https://imgur.com/a/CedCgWA), you will now have to enter the name of your spreadsheet in the options below. Then, run the cell.

[example sheet](https://docs.google.com/spreadsheets/d/105FAvXZa_E8KBOORGyvdIXhkCpoxqgJJAVn_OxCX_CA/edit?usp=sharing)



In [195]:
options = {}

# Enter your sheetname below

options["SHEETNAME"] = 'RYM AniChart 4.1'

#"2000s Movies"

## ADVANCED OPTIONS (leave default values)

options["default_delimiter"] = ". "
options["special_delimiters"] = ["\) "]
options["load_from_sheet"] = True
options["finalchar"] = "]"
options["DEFAULT_RANK_OPTION"] = "BORDA_RANK_CLASSIC"
options["remove_original_title"] = True
options["size_dependent_borda"] = False
options["vote_type"] = "ID"
# options["database_id"] = "TMDB_id"
# options["link"] = "https://www.themoviedb.org/movie/"
# options["dataname"] = 'Film (2010s)'
# options["vote_matrix_csv"] = "https://raw.githubusercontent.com/YasashiiDia/ModifiedBorda/main/data/2010s%20Movies_vote_matrix.csv"
# options["titles_csv"] = "https://raw.githubusercontent.com/YasashiiDia/ModifiedBorda/main/data/2010s%20Movies%20-%20Titles.csv"
# options["metacols"] = ['Release','Runtime','Genres','Language','Cast','Director','Producer','Writer','Director of Photography','Editor','Composer','Sound Designer','Art Direction','Production Design','Costume Design','Makeup Artist']
# options["type"] = "Film"
# options["print"] = "ID"


if options["SHEETNAME"] == 'RYM AniChart 4.1':
  options["dataname"] = 'Anime Series'
  options["SHEETNAME"] = 'RYM AniChart 4.1'
  options["finalchar"] = ""
  options["database_id"] = "AniListID"
  options["link"] = "https://anilist.co/anime/"
  options["remove_original_title"] = False
  options["vote_matrix_csv"] = "./data/RYM AniChart_ranked_vote_matrix.csv"
  options["titles_csv"] = "./data/RYM AniChart_metadata.csv"
  options["metacols"] = ['Genres','Studio','Source','Episodes','First Air Date','Last Air Date']
  options["type"] = "Series"
  options["DEFAULT_RANK_OPTION"] = "BORDA_RANK"
  options["print"] = "Title"
  options["size_dependent_borda"] = True
  options["vote_type"] = "Title"

### Step 3: Compute the results

Run the cell below.

If you get **Exception: Update titles sheet** (likely when compiling the results for the first time or adding a new voter later), **append** the printed RYM shortcuts to the ID column of the titles sheet. Match the shortcuts with the appropriate titles from RYM and copy them to the titles sheet as well (see also the [sheet instructions](https://imgur.com/a/CedCgWA)).

Once the titles sheet has been updated, re-run the cell below.

If you get some empty brackets instead of RYM shortcuts in the exception: empty the titles sheet, re-run the cell, copy the newly generated ID list and the matching titles back into the titles sheet, re-run the cell. This is a bug and will be fixed in a future update.

In [ ]:
vote_matrix, vote_matrix_all_ranked, all_user_votes, meta_df, DEFAULT_RANK = load_data_from_sheet(**options)

### Step 4: Display the results

Run the cell below. Then, click on the magic wand on the top right corner of the dataframe to get an interactive dataframe. You can sort the results by clicking on a column name.

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

DEFAULT_RANK

Run the cell below for a RYM printable version:

In [ ]:
for ID in DEFAULT_RANK.sort_values(by="Rank").index:
  votes = int(DEFAULT_RANK.loc[ID]['Votes'])
  totalscore = int(DEFAULT_RANK.loc[ID]['Score'])
  rawscore = totalscore // votes
  print(f"{DEFAULT_RANK.loc[ID]['Rank']:.0f}. {ID[0]} {votes}*{rawscore}={totalscore}")

In [ ]:
# Most no. 1
gold=all_user_votes.iloc[0].value_counts()
for i, g in enumerate(gold):
  print(gold.index[i], g)

In [ ]:
# highest pos without #1

DEFAULT_RANK.drop(gold.index,axis=0,level=1)

In [ ]:
vote_matrix

In [ ]:
all_user_votes

# Save as CSV

In [ ]:
SHEETNAME = options["SHEETNAME"]
vote_matrix.to_csv(f"/content/drive/MyDrive/{SHEETNAME}_vote_matrix.csv")
print("Saved vote matrix:", f"/content/drive/MyDrive/{SHEETNAME}_vote_matrix.csv")

meta_df.to_csv(f"/content/drive/MyDrive/{SHEETNAME}_meta_df.csv")
print("Saved meta_df:", f"/content/drive/MyDrive/{SHEETNAME}_meta_df.csv")

# AniChart Metadata

In [177]:
url = 'https://graphql.anilist.co'

query = '''
query ($id: Int, $page: Int, $perPage: Int, $search: String) {
    Page (page: $page, perPage: $perPage) {
        pageInfo {
            total
            currentPage
            lastPage
            hasNextPage
            perPage
        }
        media (id: $id, search: $search, type: ANIME) {
            id
            source
            genres
            episodes
            title {
                romaji
            }


            studios {
              edges {
                id
                isMain
                node {
                  name
                }
              }
            }


        }
    }
}
'''

In [ ]:
meta_df.head()

In [ ]:
# Query Anilist
meta_df['Genres'].astype(object) # to insert lists
meta_df = meta_df.replace(np.nan,"")

def AniQuery(meta_df,variables,title_ix,query=query):
    response = requests.post(url, json={'query': query, 'variables': variables})
    parsed = json.loads(response.text)
    dat = parsed['data']['Page']['media'][0]
    #pretty_data = json.dumps(dat, indent=4, sort_keys=True)
    #print(pretty_data,'\n')

    meta_df.loc[title_ix,'Source'] = dat['source']
    if dat['studios']['edges']:
        meta_df.loc[title_ix,'Studio'] = list(filter(lambda x:x["isMain"]==True,dat['studios']['edges']))[0]['node']['name']
    meta_df.at[title_ix, 'Genres'] = dat['genres']

    meta_df.loc[title_ix, 'AniListID'] = dat['id'] 

for i, title_ix in enumerate(meta_df.index):
    title = title_ix[1]
    if i % 50 == 0:
        print(i,title)
    variables = {'search': title}
    
    if (meta_df.loc[title_ix, 'Source'] in [np.nan,None, ''] or
        meta_df.loc[title_ix, 'Genres'] in [np.nan,None, ''] or
        meta_df.loc[title_ix, 'Studio'] in [np.nan,None, ''] or
        meta_df.loc[title_ix, 'AniListID'] in [np.nan,None, '']):

        try:
            if meta_df.loc[title_ix, 'AniListID'] != "":
              variables = {"id": meta_df.loc[title_ix, 'AniListID']}
            AniQuery(meta_df,variables,title_ix)
        except KeyError:
            print('KeyError:',title)
            continue
        except IndexError:
            print("IndexError:", title)
            continue
        except TypeError:
            print("TypeError:", title)
            continue         
meta_df.head()

In [181]:
# Flatten genres list
separator = ', '
for title in meta_df.index:
    if type(meta_df.loc[title,'Genres']) == list:
        meta_df.loc[title,'Genres'] = separator.join(meta_df.loc[title,'Genres'])


# Capitalize Source
meta_df['Source'] = meta_df['Source'].str.replace('_',' ')
meta_df['Source'] = meta_df['Source'].str.title()

In [ ]:
# Query TMDb.org

f = open("/content/drive/MyDrive/tmdb_api_key.txt", "r")
api_key = f.read()[:-1]
f.close()

for i, title_ix in enumerate(meta_df.index):
    title = title_ix[1]
    if i % 50 == 0:
        print(i,title)
    
    if (meta_df.loc[title_ix,'Episodes'] in [None, ''] or
        meta_df.loc[title_ix,'First Air Date'] in [None, ''] or
        meta_df.loc[title_ix,'Last Air Date'] in [None, ''] or 
        meta_df.loc[title_ix, 'IMGID'] in [None,''] or 
        meta_df.loc[title_ix, 'TMDbID'] in [None,'']):
        
        title_id = str(meta_df.loc[title_ix, 'TMDbID'])
        
        try:
            if title_id in [None,'','None']:
                
                r = requests.get('https://api.themoviedb.org/3/search/tv?api_key='+api_key+'&query='+title)
                parsed = json.loads(r.text)  
                
                # Make sure TMDb genre contains animation (possibly: origin country is JP)
                for j, res in enumerate(parsed['results']):
                    if 16 in res['genre_ids']: # TMDb genre ID for animation = 16
                        title_id = str(res['id'])
                        meta_df.loc[title_ix, 'TMDbID'] = title_id
                        break

                if title_id in [None,'','None']:
                    raise NameError('TitleID')

        except KeyError:
            print('KeyError1:',title)
            continue
        except IndexError:
            print('IndexError1:',title)
            continue
        except NameError:
            print('NameError1',title)
            continue     

        # Get episodes
        r = requests.get('https://api.themoviedb.org/3/tv/'+title_id+'?api_key='+api_key)
        parsed = json.loads(r.text)
        #pretty_data = json.dumps(parsed, indent=4, sort_keys=True)
        #print(pretty_data)
        
        try:
            meta_df.loc[title_ix, 'IMGID'] = parsed['poster_path']
            meta_df.loc[title_ix, 'Episodes'] = parsed['number_of_episodes']
            meta_df.loc[title_ix, 'First Air Date'] = parsed['first_air_date']
            meta_df.loc[title_ix, 'Last Air Date'] = parsed['last_air_date']
        except KeyError:
            print('KeyError2:',title,title_id)
            if title_id in [np.nan]:
              print("erere")
            print(type(title_id))
            continue
        except IndexError:
            print('IndexError2:',title)
            continue
        except NameError:
            print('NameError2',title)
            continue     

In [ ]:
meta_df.to_csv(f"/content/drive/MyDrive/{SHEETNAME}_meta_df.csv")
print("Saved meta_df:", f"/content/drive/MyDrive/{SHEETNAME}_meta_df.csv")